# OpenAI Whisper Fine-tuning

### Prepare Environment

In [1]:
!pip install datasets>=2.6.1
!pip install transformers # git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
# token =
notebook_login()

### Prepare Feature Extractor and Tokenizer

#### Load WhisperFeatureExtractor

feature extractor

연속적인 speech -sampling→ 컴퓨터가 처리할 수 있는 discrete signal

The sampling rate defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

ASR 모델의 사전 훈련된 체크포인트를 미세 조정하기 전에 모델을 사전 훈련하는 데 사용된 데이터의 샘플링 속도가 모델을 미세 조정하는 데 사용된 데이터 세트의 샘플링 속도와 일치하는지 확인하는 것이 중요합니다. Whisper는 16kHz의 샘플링 속도로 사전 학습되었습니다. 미세 조정 데이터를 16kHz로 다운샘플링해야 합니다.

In [3]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

#### Load WhisperTokenizer

Whisper 모델의 아웃풋은 vocabulary item들의 단어 중 예측된 텍스트를 나타내는 index값이다. Tokenizer는 이러한 텍스트 토큰 시퀀스와 실제 텍스트 문자열을 매핑해준다.

The tokenizer is byte-pair encoding (BPE) using UTF-8 bytes, so it can encode arbitrary unicode strings.

In [4]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe", predict_timestamps=True)

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

WhisperTokenizer는 인코딩 과정에서 'Special token'들을 부여한다. 여기에는 문장의 시작과 끝을 나타내는 토큰(전사의 시작과 끝을 나타내는 토큰을 포함한다), 언어를 나타내는 토큰, 태스크(전사, 번역 등)를 나타내는 토큰 등이 있다.

In [5]:
input_str = "안녕하세요 저는 침착맨입니다"
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False, decode_with_timestamps=True)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 안녕하세요 저는 침착맨입니다
Decoded w/ special:    <|startoftranscript|><|ko|><|transcribe|>안녕하세요 저는 침착맨입니다<|endoftext|>
Decoded w/out special: 안녕하세요 저는 침착맨입니다
Are equal:             True


#### Combine To Create a WhisperProcessor

Feature Extactor와 Tokenizer를 합쳐서 → `Wav2Vec2Processor`

샘플링, batch를 생성

훈련 과정에서 Processor와 Model이라는 두 개의 객체만 사용

In [6]:
from transformers import WhisperProcessor

processor = WhisperProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer) # , task="transcribe"

### Dataset Preprocessing

> 업로드된 데이터셋을 다운로드

In [7]:
from datasets import load_dataset, load_metric, Audio

In [8]:
dataset  = load_dataset("potatoSeop/chimsuja_dataset")['train']

# sampling rate 조절
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

Generating train split:   0%|          | 0/2521 [00:00<?, ? examples/s]

In [9]:
add_dataset = load_dataset("kresnik/zeroth_korean")['train']
add_dataset = add_dataset.cast_column("audio", Audio(sampling_rate=16_000))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for kresnik/zeroth_korean contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kresnik/zeroth_korean
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
add_dataset[0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/110/110_003_0026.flac',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/110/110_003_0026.flac',
  'array': array([ 0.00000000e+00, -3.05175781e-05,  0.00000000e+00, ...,
         -1.22070312e-04,  1.83105469e-04, -1.22070312e-04]),
  'sampling_rate': 16000},
 'text': '최태민씨는 비리를 저지르고 박 대표를 이용했다는 말이 있던데요',
 'speaker_id': 110,
 'chapter_id': 3,
 'id': '110_003_0026'}

In [10]:
import random

def sample_data_print(dataset):
  rand_int = random.randint(0, len(dataset)-1)

  print("Target text:", dataset[rand_int]["script"])
  print("Input array shape:", dataset[rand_int]["audio"]["array"].shape)
  print("Sampling rate:", dataset[rand_int]["audio"]["sampling_rate"])

데이터셋에 대하여 다음의 작업을 수행할 함수
1. 오디오 데이터를 로드하고 리샘플링
2. feature extractor를 통해 1차원 오디오 배열을 log-Mel spectrogram으로 변환
3. tokenizer를 이용해 전사 데이터를 label ids로 변환

In [11]:
def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # batch["input_length"] = len(batch["input_values"])

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["script"]).input_ids
    return batch

In [12]:
def prepare_dataset_inadd(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # batch["input_length"] = len(batch["input_values"])

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [13]:
# 데이터 전처리 함수를 각 데이터 셋에 적용
from datasets import concatenate_datasets
# 전처리된 데이터 셋 두개 합치기
preporcessed_dataset = concatenate_datasets([dataset.map(prepare_dataset, remove_columns=dataset.column_names, num_proc=None),
                                             add_dataset.map(prepare_dataset_inadd, remove_columns=add_dataset.column_names, num_proc=None)])

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Map:   0%|          | 0/22263 [00:00<?, ? examples/s]

In [11]:
# preporcessed_dataset.train_test_split(test_size=0.2)
# preporcessed_dataset

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2521
})

In [14]:
preporcessed_dataset = preporcessed_dataset.train_test_split(test_size=0.2)
train_data = preporcessed_dataset['train']
val_data = preporcessed_dataset['test']

### Prepare Training

#### Define a Data Collator

Sequence-to-sequence 발화 모델을 위한 Data collator는 input_feature와 label을 독립적으로 다룬다는 점에서 독특한 성격을 보인다. input_feature는 feature extractor로, label은 tokenizer로 다루어야 한다.

input_feature는 30초 길이로 패딩되고 고정된 차원의 log_mel spectrogram으로 변환되었으므로, 우리가 할 일은 이를 PyTorch tensor로 변환하는 것뿐이다. 이를 위해 .pad 메서드의 return_tensor=pt 인자를 사용한다. 이때 이미 패딩이 완료되었으므로 여기서 추가적인 패딩 작업이 이루어지지는 않으며, 그저 input_feature를 PyTorch tensor로 변환하기만 할 것이다.

반면, label은 아직 패딩 작업이 이루어지지 않았다. 따라서 먼저 tokenizer의 .pad 메서드를 이용해 패딩 작업을 실시할 것이다. 패딩 토큰들은 -100으로 치환되며, 따라서 이 토큰들은 모델이 loss를 계산할 때는 이용되지 않을 것이다. 그리고 이후 training 작업 동안 우리는 label sequence의 시작 부분에 있는 transcript 토큰을 잘라낼 것이다.

In [15]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

In [16]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [17]:
# 데이터 콜레이터 초기화
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

#### Evaluation Metrics

검증 데이터셋에 사용할 evaluation metrics를 정의한다. 영어의 경우 WER을 사용하지만, 한국어 데이터이므로 CER을 사용하는 것이 더 적절

In [18]:
import evaluate

metric = evaluate.load('cer')

In [19]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # pad_token을 -100으로 치환
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # metrics 계산 시 special token들을 빼고 계산하도록 설정
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

### Load a Pre-Trained Checkpoint

In [20]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to("cuda")

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

Whisper 모델은 문장의 자동 생성이 시작되기 전 모델의 출력으로 강제되는 토큰(forced_decoder_ids)이 있다. 이 token ids는 전사 언어와 zero-shot ASR 작업에 영향을 미친다. 파인 튜닝을 위해 우리는 이 ids를 None으로 바꿔주어야 한다. 우리는 모델이 정확한 언어(한국어)로 예측하고 전사하도록 훈련할 것이기 때문이다.

또한 문장의 생성 중 완전하게 억제되는 토큰들도 있다(suppress_tokens). 이 토큰들은 로그 확률을 -inf로 설정하며, 따라서 샘플링되지 않는다. 우리는 이 토큰들을 비어 있는 리스트로 치환할 것이다. 즉, 어떤 토큰도 억제되지 않는다.

In [21]:
# 파인 튜닝을 위해 Whisper 모델은 문장의 자동 생성이 시작되기 전 모델의 출력으로 강제되는 토큰(forced_decoder_ids)을 None으로 세팅
# 모델이 한국어로 예측하고
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Define the Training Arguments

마지막 단계로서, 트레이닝을 위한 모든 파라미터들을 정의해야 한다. 각각의 파라미터들은 다음과 같은 의미를 갖는다.

- output_dir : 모델의 가중치를 저장하기 위한 경로를 설정한다. 이 경로는 허깅 페이스 허브의 리포지토리 이름으로도 설정 가능하다.
- generation_max_length : 평가 작업 동안 자기회귀적으로 생성되는 토큰들의 최대 길이를 설정한다.
- save_steps : 훈련 동안, 이 파라미터에 설정한 step마다 중간 체크포인트가 비동기적으로 저장 및 업로드될 것이다.
- eval_steps : 훈련 동안, 이 파라미터에 설정한 step마다 체크포인트에 대한 평가가 이루어질 것이다.
- report_to : 훈련 로그를 어디에 저장할지를 설정한다. 'azure_ml', 'comet_ml', 'mlflow', 'neptune', 'tensorboard' 그리고 'wandb'를 지원하며, 기본값은 'tensorboard'이다.

In [22]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="potatoSeop/chimsuja",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    # max_steps=4000,  # epoch 대신 설정
    num_train_epochs=10,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=3000,
    eval_steps=3000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=True,
)


트레이닝 파라미터들의 설정이 끝났다면 트레이너를 설정한다.

In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


### Training

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


#### Model uploading to HuggingFace hub

In [23]:
kwargs = {
    "dataset_tags": "potatoSeop/chimsuja_dataset",
    "dataset": "potatoSeop/chimsuja_dataset",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: valid",
    "language": "ko",
    "model_name": "whisper-chimsuja-added",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-base",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}


In [24]:
trainer.push_to_hub(**kwargs)

events.out.tfevents.1703777630.58ba305a8dd9.376.0:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

'https://huggingface.co/potatoSeop/chimsuja/tree/main/'

In [25]:
output_dir="potatoSeop/chimsuja-added"
processor.push_to_hub(output_dir)
tokenizer.push_to_hub(output_dir)

CommitInfo(commit_url='https://huggingface.co/potatoSeop/chimsuja/commit/a3c5882dd7da8b784472008846f272256c3c7979', commit_message='Upload tokenizer', commit_description='', oid='a3c5882dd7da8b784472008846f272256c3c7979', pr_url=None, pr_revision=None, pr_num=None)

### Evaluation

#### Model Selection

In [ ]:
# 파인 튜닝한 모델을 로드
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer

model = WhisperForConditionalGeneration.from_pretrained("허브에 업로드한 모델 주소 입력").to("cuda")
model.config.forced_decoder_ids = None

processor = WhisperProcessor.from_pretrained("허브에 업로드한 모델 주소 입력")

#### Training argument 설정

inference

In [ ]:
# 방법 1
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# audio_sample = ds[3]
# speech_data = audio_sample["audio"]["array"]
# speech_file = audio_sample["file"] # used as an example for the pipeline

# inputs = processor.feature_extractor(speech_data, return_tensors="pt", sampling_rate=16_000).input_features
sample = librosa.load('/content/침착맨sample.mp3')
input_features = processor.feature_extractor(sample[0], sampling_rate=16_000, return_tensors="pt").input_features.to("cuda")

generate_ids = model.generate(input_features, task="transcribe", return_timestamps=True)
# print(generate_ids)
transcription = processor.tokenizer.decode(generate_ids[0], decode_with_timestamps=True)
transcription

'<|startoftranscript|><|ko|><|transcribe|><|0.00|> 오늘 제가 간비를 피우면서<|3.52|><|3.52|> 탁탁탁탁하고<|4.80|><|4.80|> 공촬영을 가지고<|6.46|><|6.46|> 차로 갔어요<|7.70|><|7.70|> 눈이 확 타다는데<|9.60|><|9.60|> 그...<|10.56|><|10.56|> 궁금한 냄새가 나니까<|12.78|><|12.78|> 그래서<|14.22|><|14.22|> 겨울이 되고<|15.48|><|15.48|> 이제 두 개 이제 궁금하<|17.48|><|17.48|> 시즌이 없구나<|19.04|><|19.04|> 그래서 이제<|20.08|><|20.08|> 감상이 찾아가실 게 있는데<|22.68|><|22.68|> 연기가 좀<|24.12|><|24.12|> 나오는 걸 여기서<|25.68|><|25.68|> 어?<|26.16|><|26.16|> 뭐지?<|27.12|><|27.12|> 이렇게 봤어요?<|28.92|><|28.92|><|endoftext|>'

The recommended parameters are chunk_length_s=30, stride_length_s=[6,0]. If you want to learn more about how these parameters can affect the final results, feel free to refer to the [blogpost](https://huggingface.co/blog/asr-chunking) on chunking for ASR.

In [27]:
# 방법 2 -> 추천
from transformers import pipeline
import librosa

pipe = pipeline("automatic-speech-recognition", model=model, device="cuda", tokenizer=tokenizer, feature_extractor=feature_extractor)
sample = librosa.load('/content/chickenangel.wav')

transcription = pipe(sample[0], return_timestamps=True, chunk_length_s=30, stride_length_s=[6,0], batch_size=32)
transcription

{'text': ' 아저씨가 그냥 가게 안 돼 그 순줄기 있는 오만큰한 아프게 하자 아이고 아이고 아이고 쉬는 거 어디 가요 나 어디서 많이 본 것 같아 시비에 나오지 않았나 시비에 나 오지 않았나? 너 오지 못 본 것 같은데 목소리도 많이 들어본 것 같고 어디 나왔지 아니 시미 유튜브 이벤트 뭐인데 내가 구독해 줄게요 진창맨이로 나는 그 낙시하는 사람 그게 좀 생이던데 그래서 나면 그게 좀 생이던데 내가 자주 보는 거 있어 입질의 추억이라고',
 'chunks': [{'timestamp': (0.0, 10.32),
   'text': ' 아저씨가 그냥 가게 안 돼 그 순줄기 있는 오만큰한 아프게 하자'},
  {'timestamp': (10.32, 19.08), 'text': ' 아이고 아이고 아이고 쉬는 거 어디 가요'},
  {'timestamp': (19.08, 25.62), 'text': ' 나 어디서 많이 본 것 같아 시비에 나오지 않았나'},
  {'timestamp': (24.0, 25.78), 'text': ' 시비에 나 오지 않았나?'},
  {'timestamp': (25.78, 27.7), 'text': ' 너 오지 못 본 것 같은데'},
  {'timestamp': (30.04, 33.76), 'text': ' 목소리도 많이 들어본 것 같고 어디 나왔지'},
  {'timestamp': (35.4, 37.88), 'text': ' 아니 시미'},
  {'timestamp': (41.04, 45.04), 'text': ' 유튜브 이벤트 뭐인데 내가 구독해 줄게요'},
  {'timestamp': (45.04, 46.02), 'text': ' 진창맨이로'},
  {'timestamp': (46.02, 50.28), 'text': ' 나는 그 낙시하는 사람 그게 좀 생이던데'},
  {'timestamp': (48.0, 56.28),
   'text': ' 그래서 나면 그게 좀 생이던데 내가 자주 보는 거 있어 입질의 추억이라고'}]}

In [28]:
pipe = pipeline("automatic-speech-recognition", model=model, device="cuda", tokenizer=tokenizer, feature_extractor=feature_extractor)
sample = librosa.load('/content/relaxman.wav')

transcription = pipe(sample[0], return_timestamps=True, chunk_length_s=30, stride_length_s=[6,0], batch_size=32)
transcription

{'text': ' 그 정도 빈도의 위투면 큰 게 나아요 이거 많이 컸어 택시가 됐어요 해봐 안녕하세요 떡 탑소 뭐 보인 줄까요 유튜브에서 구새끼께서 아 지금 보인인데 비비는 안 나오고 있대 예 힙을 놔 납시 누구지? 낙식도 굳이 아예 없을 수 거의 뭐 손손도 먹고 오죠 이렇게 대화하는 소강 되잖아 일단 거야 만약에 이제 대화하는 게 싫다 말이 끊기는 타임이 있어 잘 가야 돼도',
 'chunks': [{'timestamp': (0.0, 5.16), 'text': ' 그 정도 빈도의 위투면 큰 게 나아요'},
  {'timestamp': (10.56, 16.2), 'text': ' 이거 많이 컸어 택시가 됐어요 해봐 안녕하세요 떡 탑소'},
  {'timestamp': (19.2, 22.2), 'text': ' 뭐 보인 줄까요'},
  {'timestamp': (24.0, 26.0), 'text': ' 유튜브에서 구새끼께서'},
  {'timestamp': (28.0, 30.0), 'text': ' 아 지금 보인인데'},
  {'timestamp': (32.0, 34.0), 'text': ' 비비는 안 나오고 있대'},
  {'timestamp': (34.0, 36.0), 'text': ' 예 힙을 놔'},
  {'timestamp': (48.0, 50.0), 'text': ' 납시 누구지?'},
  {'timestamp': (48.0, 62.44),
   'text': ' 낙식도 굳이 아예 없을 수 거의 뭐 손손도 먹고 오죠 이렇게 대화하는 소강 되잖아'},
  {'timestamp': (62.44, 68.96),
   'text': ' 일단 거야 만약에 이제 대화하는 게 싫다 말이 끊기는 타임이 있어 잘 가야 돼도'}]}

In [ ]:
len(transcription['chunks'])

56

evaluation

In [ ]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

librispeech_test_clean = load_dataset("librispeech_asr", "clean", split="test")

processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to("cuda")

def map_to_pred(batch):
    audio = batch["audio"]
    input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['text'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

result = librispeech_test_clean.map(map_to_pred)

wer = load("wer")
print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))
